<a href="https://colab.research.google.com/github/joepareti54/joepareti54/blob/main/quantized-5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.quantization
import time
import os

# Load and normalize CIFAR10
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Define a simple model
model = torchvision.models.resnet18(pretrained=True)
model = model.to('cpu')
model.eval()
# Specify the list of modules to fuse
fuse_list = [
    ['conv1', 'bn1'],
    ['layer1.0.conv1', 'layer1.0.bn1'],
    ['layer1.0.conv2', 'layer1.0.bn2'],
    ['layer1.1.conv1', 'layer1.1.bn1'],
    ['layer1.1.conv2', 'layer1.1.bn2'],
    # ... add all the layers you want to fuse
]

# Fuse the model in place manually
model = torch.quantization.fuse_modules(model, fuse_list, inplace=True)

# Specify quantization configuration
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# Switch to evaluation mode
#model.eval()

# Prepare the model for static quantization. This inserts observers in
# the model that will observe activation tensors during calibration.
torch.quantization.prepare(model, inplace=True)

# Train the model (we're just doing one epoch for simplicity)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(1):
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Calibrate at the end of epoch
    with torch.no_grad():
        for data in trainloader:
            inputs, labels = data
            outputs = model(inputs)

# Convert the observed model to a quantized model. This does the actual quantization,
# but the model still has the quantization parameters from calibration.
torch.quantization.convert(model, inplace=True)

def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

def time_model_evaluation(model, dataloader):
    start_time = time.time()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the test images: %d %%' % (
        100 * correct / total))
    print('Evaluation time (s): ', time.time() - start_time)

# Compare the two models
print_size_of_model(model)
time_model_evaluation(model, testloader)


Files already downloaded and verified
Files already downloaded and verified
Size (MB): 11.905518


NotImplementedError: Could not run 'quantized::conv2d.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d.new' is only available for these backends: [QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

QuantizedCPU: registered at ../aten/src/ATen/native/quantized/cpu/qconv.cpp:1874 [kernel]
Quan